In [ ]:
# image url로는 request해서 불러오는 게 불가능한 상황
# selenium을 이용하여 해당 url에서 image를 다운

from selenium import webdriver
from urllib.parse import urlparse
import pandas as pd
import pyautogui
import time

# 웹 드라이버 설정
driver = webdriver.Chrome('path_to_your_chromedriver')

# CSV 파일 경로
csv_file_path = r'C:\Users\LBC\Desktop\capstone\cosmetic_info_backup.csv'

# CSV 파일 읽기
data = pd.read_csv(csv_file_path)

print("data")

for index, row in data.iterrows():
    brand_name = row['brand_name']
    product_name = row['product_name']
    image_url = row['image_url']
    
    # 쿼리 파라미터 제거
    o = urlparse(image_url)
    image_url = o.scheme + "://" + o.netloc + o.path

    # 파일 이름 만들기 (특수 문자는 공백으로 대체)
    filename = f"{brand_name}_{product_name}".replace('/', '_').replace(' ', '_')

    # 웹 사이트 열기
    driver.get(image_url)

    # 페이지 로딩 대기
    time.sleep(5)

    # 마우스 우클릭
    pyautogui.click(button='right')

    # '이미지를 다른 이름으로 저장'을 위한 아래로 키보드 입력
    pyautogui.press('down', presses=2)

    # 엔터 입력
    pyautogui.press('enter')

    # 페이지 로딩 대기
    time.sleep(2)

    # 파일명 입력
    pyautogui.hotkey('ctrl', 'a')
    pyautogui.write(filename)

    # 저장
    pyautogui.press('enter')

    # 페이지 로딩 대기
    time.sleep(2)

# 웹 드라이버 종료
driver.quit()

In [1]:
# 위 code에서 중간중간 특수문자가 포함된 product name으로 인해 오류 발생
# 해당 부분만 다시 image 다운

from selenium import webdriver
from urllib.parse import urlparse
import pandas as pd
import pyautogui
import time
import re
import numpy as np

# 웹 드라이버 설정
driver = webdriver.Chrome('path_to_your_chromedriver')

# CSV 파일 경로
csv_file_path = r'C:\Users\LBC\Desktop\capstone\cosmetic_info_backup.csv'

# CSV 파일 읽기
data = pd.read_csv(csv_file_path)

# 특수 문자를 찾는 정규식 패턴
pattern = re.compile(r'[\\/*?:"<>|]')

# 다운로드한 데이터에 대한 정보를 담는다. (0은 다운로드하지 않음, 1은 다운로드 함)
downloaded = np.zeros(len(data), dtype=int)

for index, row in data.iterrows():
    brand_name = row['brand_name']
    product_name = row['product_name']
    
    # 제품 이름에 문제의 특수 문자가 포함되어 있는지 확인
    if pattern.search(product_name):
        for i in range(2):  # 현재 행과 다음 행에서 실행
            if index + i < len(data) and downloaded[index + i] == 0:
                row = data.iloc[index + i]
                brand_name = row['brand_name']
                product_name = row['product_name']
                image_url = row['image_url']

                # 쿼리 파라미터 제거
                o = urlparse(image_url)
                image_url = o.scheme + "://" + o.netloc + o.path

                # 파일 이름 만들기 (특수 문자는 공백으로 대체)
                # 추가적으로 파일 이름으로 사용될 수 없는 문자들도 제거합니다.
                filename = re.sub(r'[\\/*?:"<>|]',"", f"{brand_name}_{product_name}")
                filename = filename.replace('/', '_').replace(' ', '_')

                # 웹 사이트 열기
                driver.get(image_url)

                # 페이지 로딩 대기
                time.sleep(5)

                # 마우스 우클릭
                pyautogui.click(button='right')

                # '이미지를 다른 이름으로 저장'을 위한 아래로 키보드 입력
                pyautogui.press('down', presses=2)

                # 엔터 입력
                pyautogui.press('enter')

                # 페이지 로딩 대기
                time.sleep(2)

                # 파일명 입력
                pyautogui.hotkey('ctrl', 'a')
                pyautogui.write(filename)

                # 저장
                pyautogui.press('enter')

                # 페이지 로딩 대기
                time.sleep(2)

                # 해당 인덱스의 데이터를 다운로드했음을 표시
                downloaded[index + i] = 1

# 웹 드라이버 종료
driver.quit()

C:\Users\LBC\AppData\Local\Temp\ipykernel_49228\2619957883.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('path_to_your_chromedriver')


In [2]:
# image의 확장자를 jpg로 통일
# 오류가 발생하는 image는 print

import os
from PIL import Image
import pillow_avif

def convert_to_jpg(input_file_path, output_directory):
    # 입력된 파일 이름에서 확장자를 제외한 부분을 가져옵니다.
    base_name = os.path.basename(input_file_path)
    file_name_without_extension, _ = os.path.splitext(base_name)
    
    # 출력 파일 경로를 만듭니다. 확장자는 .jpg로 설정합니다.
    output_file_path = os.path.join(output_directory, f"{file_name_without_extension}.jpg")
    
    try:
        img = Image.open(input_file_path)
        img.save(output_file_path, 'JPEG')
    except IOError:
        print(f"Cannot convert image: {input_file_path}")

def process_directory(input_directory, output_directory):
    # 입력 디렉토리에서 모든 파일을 순회합니다.
    for root, _, files in os.walk(input_directory):
        for file in files:
            # 파일 확장자가 avif, png, jpg인 경우에만 변환을 수행합니다.
            if file.lower().endswith((".avif", ".png", ".jpg", ".webp")):
                input_file_path = os.path.join(root, file)
                convert_to_jpg(input_file_path, output_directory)

process_directory(r'C:\Users\LBC\Desktop\temp', r'C:\Users\LBC\Desktop\temp2')

Cannot convert image: C:\Users\LBC\Desktop\temp\DedCool_01_Taunt_Air_Freshener.png
Cannot convert image: C:\Users\LBC\Desktop\temp\Dior_Dior_Forever_Skin_Glow_Foundation_SPF_15.webp
Cannot convert image: C:\Users\LBC\Desktop\temp\Dr._Dennis_Gross_Skincare_All-Physical_Ultimate_Defense_Broad_Spectrum_Sunscreen_SPF_50_PA++++.webp
Cannot convert image: C:\Users\LBC\Desktop\temp\Ellis_Brooklyn_VANILLA_MILK_Eau_de_Parfum_Travel_Spray.jpg
Cannot convert image: C:\Users\LBC\Desktop\temp\Glow_Recipe_Avocado_Fine_Line_Eye_Cream_with_Retinol.png
Cannot convert image: C:\Users\LBC\Desktop\temp\goop_GOOPGENES_Clean_Nourishing_Lip_Balm_Trio.jpg
Cannot convert image: C:\Users\LBC\Desktop\temp\Krastase_Genesis_Heat_Protecting_Leave-In_Treatment_for_Weakened_Hair.webp
Cannot convert image: C:\Users\LBC\Desktop\temp\Krastase_Genesis_Strengthening_Conditioner.webp
Cannot convert image: C:\Users\LBC\Desktop\temp\Krastase_Symbiose_Antidandruff_Hydrating_Shampoo.webp
Cannot convert image: C:\Users\LBC\Desk

In [1]:
# 브랜드별로 folder를 만들고 해당하는 image를 집어넣는다.

import os
import glob
import pandas as pd
import shutil

def create_and_populate_folders(csv_path, src_dir, dst_dir):
    # CSV 파일 읽기
    df = pd.read_csv(csv_path)
    brand_names = df['brand_name'].unique()
    
    for brand in brand_names:
        # 공백을 '_'로 대체
        safe_brand = brand.replace(' ', '_')

        # 출력 디렉토리 경로를 만듭니다.
        dir_path = os.path.join(dst_dir, safe_brand)
        
        # 해당 경로의 디렉토리가 없을 경우, 디렉토리를 생성합니다.
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)
        
        # 해당 브랜드 이름으로 시작하는 모든 파일을 찾아서 새로운 디렉토리로 복사합니다.
        src_files = glob.glob(os.path.join(src_dir, f"{safe_brand}*"))
        for file_path in src_files:
            shutil.copy(file_path, dir_path)
        
        # 복사 후 파일 개수를 비교
        dst_files = glob.glob(os.path.join(dir_path, '*'))
        if len(src_files) != len(dst_files):
            print(f"브랜드 이름: {brand}, 원본 파일 개수: {len(src_files)}, 복사된 파일 개수: {len(dst_files)}")

# CSV 파일 경로
csv_file_path = r'C:\Users\LBC\Desktop\capstone\cosmetic_info_backup.csv'
            
# 함수 호출
create_and_populate_folders(csv_file_path, r'C:\Users\LBC\Desktop\temp2', r'C:\Users\LBC\Desktop\images')

In [6]:


import pandas as pd

def count_elements(csv_path):
    # CSV 파일 읽기
    df = pd.read_csv(csv_path)
    
    # DataFrame 출력 설정 변경: 최대 행 수 제한 해제
    pd.set_option('display.max_rows', None)
    
    # 각 brand_name의 수를 계산하고 출력합니다.
    brand_counts = df['brand_name'].value_counts().sort_index()
    print(brand_counts)

# 함수 호출
count_elements(r'C:\Users\LBC\Desktop\capstone\cosmetic_info_backup.csv')

19-69                                 14
54 Thrones                             4
ABBOTT                                16
AERIN                                 20
ALTERNA Haircare                      43
Acqua di Parma                        20
Algenist                              32
Alpha-H                                9
Ami Colé                               9
Anastasia Beverly Hills               94
Aquis                                  1
Armani Beauty                         60
Artist Couture                        11
Atelier Cologne                        2
Augustinus Bader                      22
Azzaro                                 1
BASMA                                  1
BREAD BEAUTY SUPPLY                   15
BURBERRY                              28
BeautyBio                             29
Benefit Cosmetics                     72
Bio Ionic                             12
Biossance                             26
Blinc                                  4
Bobbi Brown     

In [8]:
# 각 brand의 folder에 image가 재대로 들어갔는지 확인

import os
import pandas as pd
import glob

# CSV 파일 읽기
csv_path = r"C:\Users\LBC\Desktop\capstone\cosmetic_info_backup.csv"
df = pd.read_csv(csv_path)

# Clarins의 제품 이름 추출, 파일 이름으로 변환 시 "Clarins_"을 앞에 붙임
clarins_products = df[df['brand_name'] == 'FaceGym']['product_name'].apply(lambda x: "FaceGym_" + x.replace(' ', '_')).unique()

# 디렉토리의 모든 파일 이름을 가져옵니다.
dir_path = r"C:\Users\LBC\Desktop\images\FaceGym"
file_paths = glob.glob(os.path.join(dir_path, "*"))
file_names = [os.path.splitext(os.path.basename(fp))[0] for fp in file_paths]

# Clarins 제품 이름 중에서 파일이 없는 것을 찾습니다.
missing_files = [product for product in clarins_products if product not in file_names]

# 결과를 출력합니다.
print("Missing Files:", missing_files)


Missing Files: ['FaceGym_Faceshot™_Vitamin_Ampoules_Refill']


In [12]:
# Nécessaire 같이 불어를 사용하여 재대로 이름이 저장되지 않은 경우를 수정

import os
import glob

# 디렉토리의 모든 파일 이름을 가져옵니다.
dir_path = r"C:\Users\LBC\Desktop\images\Nécessaire"
file_paths = glob.glob(os.path.join(dir_path, "*"))

for file_path in file_paths:
    base_name = os.path.basename(file_path)
    if "Ncessaire" in base_name:
        new_name = base_name.replace("Ncessaire", "Nécessaire")
        new_file_path = os.path.join(dir_path, new_name)
        os.rename(file_path, new_file_path)


In [15]:
# 분류 후 더 이상 필요없는 brand name을 image file에서 제거

import os
import glob

# 상위 디렉토리 경로
dir_path = r"C:\Users\LBC\Desktop\images"

# 상위 디렉토리 안의 모든 하위 디렉토리를 가져옵니다.
sub_dirs = [x[0] for x in os.walk(dir_path)][1:]

# 모든 하위 디렉토리를 돌면서 파일 이름을 수정합니다.
for sub_dir in sub_dirs:
    brand_name = os.path.basename(sub_dir)
    file_paths = glob.glob(os.path.join(sub_dir, "*"))
    
    for file_path in file_paths:
        base_name = os.path.basename(file_path)
        # 파일 이름에서 브랜드 이름 부분을 제거합니다.
        new_name = base_name.replace(brand_name + "_", "")
        new_file_path = os.path.join(sub_dir, new_name)
        os.rename(file_path, new_file_path)


In [16]:
# 각 image를 위한 folder를 만들고 그 안에 image insert

import os
import glob
import shutil

# 상위 디렉토리 경로
dir_path = r"C:\Users\LBC\Desktop\images"

# 상위 디렉토리 안의 모든 하위 디렉토리를 가져옵니다.
sub_dirs = [x[0] for x in os.walk(dir_path)][1:]

# 모든 하위 디렉토리를 돌면서 파일 이름을 수정합니다.
for sub_dir in sub_dirs:
    file_paths = glob.glob(os.path.join(sub_dir, "*"))
    
    for file_path in file_paths:
        base_name = os.path.basename(file_path)
        # 확장자 제거
        file_name_without_extension, _ = os.path.splitext(base_name)
        # 새로운 디렉토리 경로 생성
        new_dir_path = os.path.join(sub_dir, file_name_without_extension)
        
        # 새로운 디렉토리 생성
        os.makedirs(new_dir_path, exist_ok=True)

        # 파일을 새로운 디렉토리로 이동
        shutil.move(file_path, new_dir_path)
